In [1]:
import os
import sys
import numpy as np
import pyphone
print(os.getcwd())
print(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.getcwd()))
import epitran
from datasets import load_from_disk
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from functools import  lru_cache
import re

/home/toure215/BERT_phonetic/test
/home/toure215/BERT_phonetic


In [2]:
dataset = load_from_disk("/home/toure215/BERT_phonetic/DATASETS/verses/hf_rhymes")
sample = dataset['train'][:10]
sample

{'id': [106664,
  184645,
  133811,
  140929,
  108312,
  33067,
  129173,
  118678,
  139076,
  3648],
 'sentence1': ['because it from no spurious passion came',
  'she tells him he should hope the best',
  'see the tall obeliscs from memphis old',
  'where i at last shall rest my weary head',
  'seagrass and spreading wrack are seen below',
  "all matter lives and shews its maker's power",
  "by laws eternal to th' aerial kind",
  'with their own force his panting breast they arm',
  'grutch not of mammon and his leaven',
  'she bids each slumbering energy awake'],
 'sentence2': ['in that kind hour thy fatal letter came',
  'one that still needs his leading string and bib',
  'makes truth and discretion the guide of her life',
  "the fairfac'd nuns to fornication draw",
  'what strange disorder prompts these thoughts to glow',
  'softly the angelus sounded and over the roofs of the village',
  'backgoback oshaugodaya',
  'left me each tender fond affection warm',
  'with ceaseless pr

In [3]:
for sentence1, sentenc2, label in zip(sample['sentence1'], sample['sentence2'], sample['label']):
    print(f"{sentence1} - {sentenc2} - {label}")

because it from no spurious passion came - in that kind hour thy fatal letter came - 1
she tells him he should hope the best - one that still needs his leading string and bib - 0
see the tall obeliscs from memphis old - makes truth and discretion the guide of her life - 0
where i at last shall rest my weary head - the fairfac'd nuns to fornication draw - 0
seagrass and spreading wrack are seen below - what strange disorder prompts these thoughts to glow - 1
all matter lives and shews its maker's power - softly the angelus sounded and over the roofs of the village - 0
by laws eternal to th' aerial kind - backgoback oshaugodaya - 0
with their own force his panting breast they arm - left me each tender fond affection warm - 1
grutch not of mammon and his leaven - with ceaseless prayers the whole artillery given - 1
she bids each slumbering energy awake - one deep obliterating draught of lethe take - 1


In [4]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/home/toure215/miniconda3/envs/bert/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding=False, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['sentence1', 'sentence2', 'id'])
tokenized_dataset

Map:   0%|          | 0/15921 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 143280
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 15921
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 39801
    })
})

In [6]:
datacollector = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
training_args = TrainingArguments(
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=3,
    logging_strategy='no',
    eval_strategy='steps',
    output_dir='/tmp/test',
    save_strategy='no',
    fp16=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=datacollector,
)

/home/toure215/miniconda3/envs/bert/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [9]:
trainer.train()

  0%|          | 0/1680 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.021815825253725052, 'eval_runtime': 1.661, 'eval_samples_per_second': 9585.335, 'eval_steps_per_second': 37.93, 'epoch': 0.89}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.012306553311645985, 'eval_runtime': 1.7442, 'eval_samples_per_second': 9127.834, 'eval_steps_per_second': 36.119, 'epoch': 1.79}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.012638099491596222, 'eval_runtime': 1.6892, 'eval_samples_per_second': 9425.38, 'eval_steps_per_second': 37.297, 'epoch': 2.68}
{'train_runtime': 122.5703, 'train_samples_per_second': 3506.886, 'train_steps_per_second': 13.706, 'train_loss': 0.028571730568295435, 'epoch': 3.0}


TrainOutput(global_step=1680, training_loss=0.028571730568295435, metrics={'train_runtime': 122.5703, 'train_samples_per_second': 3506.886, 'train_steps_per_second': 13.706, 'total_flos': 7220745803315520.0, 'train_loss': 0.028571730568295435, 'epoch': 3.0})

In [10]:
predictions = trainer.predict(tokenized_dataset['test'])
preds, labels = predictions.predictions, predictions.label_ids
print(preds.shape, labels.shape)
preds = np.argmax(preds, axis=1)
print((preds == labels).mean())

  0%|          | 0/156 [00:00<?, ?it/s]

(39801, 2) (39801,)
0.9961810004773749


In [11]:
phonetic_dataset = load_from_disk("/home/toure215/BERT_phonetic/DATASETS/verses/phonetic_hf_rhymes")
sample2 = phonetic_dataset['train'][:20]
for sentence1, sentenc2, label in zip(sample2['sentence1'], sample2['sentence2'], sample2['label']):
    print(f"{sentence1} -- {sentenc2} -- {label}")

bIkOz It fr\Vm now spjUr\i@s p{S@n kejm -- In D{t kajnd awr\= Daj fejt@l lEtr\= kejm -- 1
Si tElz hIm hi SUd howp D@ bEst -- wVn D{t stIl nidz hIz lidIN str\IN {nd bIb -- 0
si D@ tOl AbElIsks fr\Vm mEmf@s owld -- mejks tr\uT {nd dIskr\ES@n D@ gajd Vv hr\= lajf -- 0
wEr\ aj {t l{st S{l r\Est maj wIr\i hEd -- D@ fEr\f{k  di nVnz t@ fOr\n@kejS@n dr\O -- 0
sigr\@s {nd spr\EdIN r\{k Ar\ sin bIlow -- wVt str\ejndZ dIsOr\dr\= pr\Ampts Diz TOts t@ glow -- 1
Ol m{tr\= lajvz {nd Sjuz Its mejkr\=  Es pawr\= -- sOftli D@ {ndZ@l@s sawnd@d {nd owvr\= D@ r\ufs Vv D@ vIl@dZ -- 0
baj lOz Itr\=n@l t@ tiejtS  Er\i@l kajnd -- b{kgowb{k OSOgowdaj@ -- 0
wID DEr\ own fOr\s hIz p{ntIN br\Est Dej Ar\m -- lEft mi itS tEndr\= fAnd @fEkS@n wOr\m -- 1
gr\VtS nAt Vv m{m@n {nd hIz liv@n -- wID sislIs pr\Er\z D@ howl Ar\tIlr\=i gIv@n -- 1
Si bIdz itS slVmbr\=IN Enr\=dZi @wejk -- wVn dip @blitr\=ejtIN dr\{ft Vv lED tejk -- 1
D{t hilz D@ wawnd {nd kjUr\z nAt D@ dIsgr\ejs -- gAz  di An Daj blVS@z Ar\m  di wID Evr\=i gr\

In [8]:
"""
Python wrapper for the website: https://www.homophone.com/
Gets the homophones of a word.
"""

from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from typing import Dict, List
import re

class Pyphones:
    
    def __init__(self, word):
        self.word = word
        self.url = "https://www.homophone.com/search?page={}&type=&q={}"
        self.homophones = {self.word: []}
        
    def get_the_page(self, page_no=1):
        """
        Get the page content.

        Returns
            str: the content of the page.
        """
        url = self.url.format(page_no, self.word)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        return soup

    def get_the_page_nos(self):
        """
        Get the total number of pages

        Returns
            int: the total number of the pages.
        """
        soup = self.get_the_page()
        pages = soup.find_all('div', attrs={'class':'col-sm-9'})
        if not pages:
            return 0
        total_pages = pages[0].find('h5').text.split('/')[-1].strip()
        return int(total_pages)

    def get_the_homophones(self):
        """
        Get the homophones of the word.

        Returns
            dict: {word: [list_of_homophones]} against each word.
        """
        total_pages = self.get_the_page_nos()
        for ix in range(total_pages):
            page_no = ix + 1
            soup = self.get_the_page(page_no)
            raw_homophones = soup.find_all('div', attrs={'class': 'well well-lg'})
            for elem in range(len(raw_homophones)):
                raw_homophones_2 = raw_homophones[elem].find_all('a', attrs={'class': 'btn word-btn'})
                list_of_homophones = list(raw_homophones_2)
                if any(list_of_homophones):
                    local_homophones = []
                    for tag_of_homophone in list_of_homophones:
                        homophone = tag_of_homophone.text
                        local_homophones.append(homophone)
                    self.homophones[self.word].append(local_homophones)

        return self.homophones

In [13]:
py = Pyphones("rough")
homophones = py.get_the_homophones()
print(homophones)

{'rough': [['borough', 'burro', 'burrow'], ['rough', 'ruff'], ['through', 'threw', 'thru']]}


In [17]:
for v in homophones["rough"]:
    print(v)

print(homophones.values())

['borough', 'burro', 'burrow']
['rough', 'ruff']
['through', 'threw', 'thru']
dict_values([[['borough', 'burro', 'burrow'], ['rough', 'ruff'], ['through', 'threw', 'thru']]])
